# Import / 라이브러리 호출

In [1]:
import pandas as pd
import numpy as np
import random
import os

# Fixed RandomSeed / 랜덤시드 고정

매번 고정된 결과를 얻기 위해서 사용합니다.  
seed를 고정하지 않는다면 같은 코드라도 매번 다른 결과가 나오게됩니다.  
항상 동일한 결과를 얻기 위해서 사용합니다.


In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# Data Load / 데이터 불러오기

먼저 pandas 라이브러리의 read_csv() 함수를 이용해 분석에 사용할 데이터를 불러옵니다.

In [3]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [4]:
train.head()

,ID,월,일,측정 시간대,섭씨 온도(°⁣C),절대 온도(K),이슬점 온도(°C),상대 습도 (%),대기압(mbar),포화 증기압(mbar),실제 증기압(mbar),증기압 부족량(mbar),수증기 함량 (g/kg),공기 밀도 (g/m**3),풍향 (deg),풍속 (m/s)
0,TRAIN_00000,7,2,저녁,13.97,287.78,9.84,76.1,992.08,15.98,12.16,3.82,7.66,1198.06,155.60,1.61
1,TRAIN_00001,8,21,오전,16.94,290.85,12.14,73.3,991.07,19.33,14.17,5.16,8.94,1183.67,177.00,1.68
2,TRAIN_00002,11,1,저녁,9.76,283.84,5.40,74.2,988.71,12.10,8.98,3.12,5.67,1213.22,146.20,0.73
3,TRAIN_00003,12,28,오전,5.27,277.30,2.71,83.5,1014.25,8.89,7.43,1.47,4.57,1265.48,264.50,2.71
4,TRAIN_00004,9,26,오후,17.35,290.86,12.68,74.0,995.77,19.84,14.68,5.16,9.22,1187.40,19.34,1.00


# 데이터프레임 정보 출력

train 데이터프레임은 총 26129개의 행(RangeIndex)과 16개의 열(Data columns)로 구성되어 있습니다.   
각 컬럼(Column)의 이름과 데이터 타입(Dtype)을 보여주며, Non-Null Count 열은 결측값이 아닌 값의 개수를 보여줍니다.   

수치형 데이터가 12개(float64)와 2개(int64)의 컬럼으로 구성되어 있으며, object 타입의 컬럼도 2개 있습니다.    
따라서 'ID'와 '측정 시간대' 컬럼이 문자열 데이터라는 것을 알 수 있습니다.   

마지막으로, memory usage: 3.2+ MB는 데이터프레임이 메모리에서 사용하는 용량을 보여주고 있습니다.

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36581 entries, 0 to 36580
Data columns (total 16 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              36581 non-null  object 
 1   월               36581 non-null  int64  
 2   일               36581 non-null  int64  
 3   측정 시간대          36581 non-null  object 
 4   섭씨 온도(°⁣C)      36581 non-null  float64
 5   절대 온도(K)        36581 non-null  float64
 6   이슬점 온도(°C)      36581 non-null  float64
 7   상대 습도 (%)       36581 non-null  float64
 8   대기압(mbar)       36581 non-null  float64
 9   포화 증기압(mbar)    36581 non-null  float64
 10  실제 증기압(mbar)    36581 non-null  float64
 11  증기압 부족량(mbar)   36581 non-null  float64
 12  수증기 함량 (g/kg)   36581 non-null  float64
 13  공기 밀도 (g/m**3)  36581 non-null  float64
 14  풍향 (deg)        36581 non-null  float64
 15  풍속 (m/s)        36581 non-null  float64
dtypes: float64(12), int64(2), object(2)
memory usage: 4.5+ MB


# Feature & Target Split / 독립변수(X), 종속변수(y)로 나누기

모델을 학습하기 위해서는 독립변수(X)와 종속변수(y)로 나누어야 합니다.  
독립변수는 Feature, 종속변수는 Target이라고 불리기도 합니다.  

- Feature : 예측을 위해 활용되는 데이터  
- Target : 예측하고자 하는 대상  

(이 때, 분석에 활용하지 않는 데이터인 'id'는 제거하겠습니다.)

In [6]:
# train_x는 독립변수이므로 종속변수(풍속 (m/s))를 제거합니다. 
# 또한 target 이외의 분석에 활용하지 않는 데이터(id)를 제거합니다.
train_x = train.drop(columns=['ID', '풍속 (m/s)'], axis = 1)

# train_y는 종속변수로 값을 설정합니다.
train_y = train['풍속 (m/s)']

In [7]:
# train에서와 마찬가지로 분석에 활용하지 않는 데이터(id)를 제거합니다.
test_x = test.drop(columns=['ID'])

# Data Pre-processing

데이터 전처리란 무엇일까요?  

결측치 처리, 이상치 제거, 데이터 단위 변환, 데이터 분포 변환 등 데이터를 정확하게 분석하기 위해 먼저 데이터에 여러 가지 처리를 해주는 것입니다.  
전처리를 함으로써 데이터 분석이 가능하도록 하며, 데이터를 합치거나 나눠서 더 정확한 정보를 갖도록 해주기 때문에   
전처리 과정은 데이터 분석에 있어서 반드시 필요한 부분입니다.    
데이터를 계산하는 컴퓨터는 오로지 숫자(정수, 실수)만을 인식하기 때문입니다.  
그래서 사람이 사용하는 한국어나 영어와 같은 문자나 비어있는 값 (결측치) 등을 숫자로 변경해 주어야 합니다.  

(이번 데이터에는 결측치가 존재하지 않습니다.)

In [8]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
le = le.fit(train_x['측정 시간대'])
train_x['측정 시간대'] = le.transform(train_x['측정 시간대'])
test_x['측정 시간대'] = le.transform(test_x['측정 시간대'])

# Regression Model Definition / 회귀 모델 정의

In [9]:
from sklearn.tree import DecisionTreeRegressor

dt_clf = DecisionTreeRegressor(random_state=11) 

# Model Fit / 모델 학습

In [10]:
dt_clf.fit(train_x , train_y) 

DecisionTreeRegressor(random_state=11)

# Prediction / 예측

In [11]:
dt_pred = dt_clf.predict(test_x) 

# Submit / 제출

In [12]:
submission = pd.read_csv('./sample_submission.csv')

In [13]:
submission['풍속 (m/s)'] = dt_pred
submission.head()

,ID,풍속 (m/s)
0,TEST_00000,2.49
1,TEST_00001,0.98
2,TEST_00002,2.25
3,TEST_00003,1.11
4,TEST_00004,1.52


In [14]:
submission.to_csv('submission.csv', index=  False)